# CTU-Net

## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, backend, Model
from tensorflow.keras.applications import EfficientNetB4, EfficientNetV2S

## Block functions

In [2]:
def DB_block(input1, num_filters):
    x = layers.BatchNormalization()(input1)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    c1 = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    
    x = layers.concatenate([input1,c1]) 
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (3, 3), padding="same")(x)
    
    x = layers.concatenate([input1,x,c1])
    return x

def DBB_block(input1, input2, num_filters):
    x = DB_block(input1, num_filters)
    
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(x)
    x = layers.Activation("relu")(x)
    x = layers.BatchNormalization()(x)
    
    # Rescale input2 to have the same filter dimension as x #
    input2 = layers.Conv2D(num_filters, (1, 1), padding="same")(input2)
    ###################################################
    x = layers.Add()([x, input2])
    
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    # Reduce size of x to match its original size before Conv2DTranspose#
    x = layers.MaxPooling2D((2,2))(x)
    ###################################################
    return x

def SCAB_block(input1, input2, num_filters, final=False):
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(input1)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.Activation("sigmoid")(x)
    
    if final:
        input2 = layers.Conv2D(num_filters, (1, 1), padding="same")(input2)
        
    x = layers.Multiply()([x, input2])
    
    x = layers.concatenate([input1,x])
    
    c1 = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.GlobalAveragePooling2D(keepdims=True)(c1)
    
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    x = layers.Activation("relu")(x)
    x = layers.Conv2D(num_filters, (1, 1), padding="same")(x)
    
    x = layers.Activation("sigmoid")(x)
    
    x = layers.Multiply()([x, c1])
    
    x = layers.Add()([x, c1])
    return x

In [3]:
def conv_block(input, num_filters):
    x = layers.Conv2D(num_filters, 3, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def deconv_block(input, num_filters):
    x = layers.Conv2DTranspose(num_filters, 1, padding="same")(input)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def decoder_block(input, skip_output, num_filters, final=False):
    x = conv_block(input, num_filters)
    x = layers.Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = SCAB_block(x, skip_output, num_filters, final)
    x = DB_block(x, num_filters)
    return x

def bottleneck(input):
    x = layers.MaxPooling2D((2,2))(input)
    #x = layers.GlobalAveragePooling2D()(x)
    return x

## EfficientNetB4

In [4]:
def EfficientNetB4_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    EffNetB4 = EfficientNetB4(include_top=False, weights="imagenet", input_tensor=inputs)

    # Encoder
    res0 = EffNetB4.get_layer('rescaling_1').output  # 512 x 512
    res1 = EffNetB4.get_layer('block2a_expand_activation').output  # 256 x 256
    res2 = EffNetB4.get_layer('block3a_expand_activation').output  # 128 x 128
    res3 = EffNetB4.get_layer('block4a_expand_activation').output  # 64 x 64
    res4 = EffNetB4.get_layer('block6a_expand_activation').output  # 32 x 32
    
    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="EfficientNetB4_CTU-Net")
    return model


In [5]:
model = EfficientNetB4_CTUnet()

In [6]:
model.summary()

Model: "EfficientNetB4_CTU-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 512, 512, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 512, 512, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['normalizat

 block2a_expand_conv (Conv2D)   (None, 256, 256, 14  3456        ['block1b_add[0][0]']            
                                4)                                                                
                                                                                                  
 block2a_expand_bn (BatchNormal  (None, 256, 256, 14  576        ['block2a_expand_conv[0][0]']    
 ization)                       4)                                                                
                                                                                                  
 block2a_expand_activation (Act  (None, 256, 256, 14  0          ['block2a_expand_bn[0][0]']      
 ivation)                       4)                                                                
                                                                                                  
 block2a_dwconv_pad (ZeroPaddin  (None, 257, 257, 14  0          ['block2a_expand_activation[0][0]
 g2D)     

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_dwconv (DepthwiseConv2  (None, 128, 128, 19  1728       ['block2c_expand_activation[0][0]
 D)                             2)                               ']                               
                                                                                                  
 block2c_b

                                2)                                                                
                                                                                                  
 block3a_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block3a_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block3a_expand_activation (Act  (None, 128, 128, 19  0          ['block3a_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block3a_dwconv_pad (ZeroPaddin  (None, 131, 131, 19  0          ['block3a_expand_activation[0][0]
 g2D)                           2)                               ']                               
          

                                                                                                  
 block3c_dwconv (DepthwiseConv2  (None, 64, 64, 336)  8400       ['block3c_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3c_bn (BatchNormalization  (None, 64, 64, 336)  1344       ['block3c_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3c_activation (Activation  (None, 64, 64, 336)  0          ['block3c_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block3c_s

...

                                                                                                  
 concatenate_14 (Concatenate)   (None, 64, 64, 512)  0           ['add_5[0][0]',                  
                                                                  'conv2d_49[0][0]']              
                                                                                                  
 batch_normalization_34 (BatchN  (None, 64, 64, 512)  2048       ['concatenate_14[0][0]']         
 ormalization)                                                                                    
                                                                                                  
 activation_46 (Activation)     (None, 64, 64, 512)  0           ['batch_normalization_34[0][0]'] 
                                                                                                  
 conv2d_50 (Conv2D)             (None, 64, 64, 256)  131328      ['activation_46[0][0]']          
          

 conv2d_58 (Conv2D)             (None, 128, 128, 12  16512       ['activation_54[0][0]']          
                                8)                                                                
                                                                                                  
 batch_normalization_39 (BatchN  (None, 128, 128, 12  512        ['conv2d_58[0][0]']              
 ormalization)                  8)                                                                
                                                                                                  
 activation_55 (Activation)     (None, 128, 128, 12  0           ['batch_normalization_39[0][0]'] 
                                8)                                                                
                                                                                                  
 conv2d_59 (Conv2D)             (None, 128, 128, 12  147584      ['activation_55[0][0]']          
          

                                                                                                  
 conv2d_67 (Conv2D)             (None, 1, 1, 64)     4160        ['activation_62[0][0]']          
                                                                                                  
 activation_63 (Activation)     (None, 1, 1, 64)     0           ['conv2d_67[0][0]']              
                                                                                                  
 multiply_7 (Multiply)          (None, 256, 256, 64  0           ['activation_63[0][0]',          
                                )                                 'conv2d_65[0][0]']              
                                                                                                  
 add_7 (Add)                    (None, 256, 256, 64  0           ['multiply_7[0][0]',             
                                )                                 'conv2d_65[0][0]']              
          

                                                                                                  
 conv2d_75 (Conv2D)             (None, 512, 512, 32  2176        ['concatenate_1[0][0]']          
                                )                                                                 
                                                                                                  
 multiply_8 (Multiply)          (None, 512, 512, 32  0           ['activation_71[0][0]',          
                                )                                 'conv2d_75[0][0]']              
                                                                                                  
 concatenate_22 (Concatenate)   (None, 512, 512, 64  0           ['activation_69[0][0]',          
                                )                                 'multiply_8[0][0]']             
                                                                                                  
 conv2d_76

 conv2d_transpose_9 (Conv2DTran  (None, 512, 512, 16  272        ['activation_78[0][0]']          
 spose)                         )                                                                 
                                                                                                  
 batch_normalization_55 (BatchN  (None, 512, 512, 16  64         ['conv2d_transpose_9[0][0]']     
 ormalization)                  )                                                                 
                                                                                                  
 activation_79 (Activation)     (None, 512, 512, 16  0           ['batch_normalization_55[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_84 (Conv2D)             (None, 512, 512, 16  2320        ['activation_79[0][0]']          
          

## EfficientNetV2S

In [6]:
def EfficientNetV2S_CTUnet(input_shape=(512, 512, 3)):
    # Input
    inputs = layers.Input(input_shape)

    # Loading pre trained model
    EffNetV2S = EfficientNetV2S(include_top=False, weights="imagenet", input_tensor=inputs)
    print(EffNetV2S.summary())
    
    # Encoder
    res0 = EffNetV2S.get_layer('rescaling').output  # 512 x 512
    res1 = EffNetV2S.get_layer('block1b_add').output  # 256 x 256
    res2 = EffNetV2S.get_layer('block2d_add').output  # 128 x 128
    res3 = EffNetV2S.get_layer('block4a_expand_activation').output  # 64 x 64
    res4 = EffNetV2S.get_layer('block6a_expand_activation').output  # 32 x 32
    
    # Skip connection blocks
    skip0 = DB_block(res0, 32)
    skip1 = DBB_block(res1, skip0, 64)
    skip2 = DBB_block(res2, skip1, 128)
    skip3 = DBB_block(res3, skip2, 256)
    skip4 = DBB_block(res4, skip3, 512)
    
    # Bottleneck
    b1 = bottleneck(res4)  # 16 x 16

    # Decoder
    d1 = decoder_block(b1, skip4, 512)  # 32 x 32
    d2 = decoder_block(d1, skip3, 256)    # 64 x 64
    d3 = decoder_block(d2, skip2, 128)   # 128 x 128
    d4 = decoder_block(d3, skip1, 64)   # 256 x 256
    d5 = decoder_block(d4, skip0, 32, True)   # 512 x 512

    # Output
    outputs = conv_block(d5, 16)
    outputs = deconv_block(outputs, 16)
    outputs = conv_block(outputs, 16)
    outputs = layers.Conv2D(1, 1, padding="same", activation="sigmoid")(outputs)

    model = Model(inputs, outputs, name="EfficientNetV2S_CTU-Net")
    
    return model

In [7]:
model1 = EfficientNetV2S_CTUnet()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 512, 512, 3)  0           ['input_2[0][0]']                
                                                                                                  
 stem_conv (Conv2D)             (None, 256, 256, 24  648         ['rescaling_1[0][0]']            
                                )                                                                 
                                                                                   

                                                                                                  
 block2c_expand_bn (BatchNormal  (None, 128, 128, 19  768        ['block2c_expand_conv[0][0]']    
 ization)                       2)                                                                
                                                                                                  
 block2c_expand_activation (Act  (None, 128, 128, 19  0          ['block2c_expand_bn[0][0]']      
 ivation)                       2)                                                                
                                                                                                  
 block2c_project_conv (Conv2D)  (None, 128, 128, 48  9216        ['block2c_expand_activation[0][0]
                                )                                ']                               
                                                                                                  
 block2c_p

                                                                                                  
 block3c_project_bn (BatchNorma  (None, 64, 64, 64)  256         ['block3c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block3c_drop (Dropout)         (None, 64, 64, 64)   0           ['block3c_project_bn[0][0]']     
                                                                                                  
 block3c_add (Add)              (None, 64, 64, 64)   0           ['block3c_drop[0][0]',           
                                                                  'block3b_add[0][0]']            
                                                                                                  
 block3d_expand_conv (Conv2D)   (None, 64, 64, 256)  147456      ['block3c_add[0][0]']            
          

                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 32)     16416       ['block4b_se_reshape[0][0]']     
                                                                                                  
 block4b_se_expand (Conv2D)     (None, 1, 1, 512)    16896       ['block4b_se_reduce[0][0]']      
                                                                                                  
 block4b_se_excite (Multiply)   (None, 32, 32, 512)  0           ['block4b_activation[0][0]',     
                                                                  'block4b_se_expand[0][0]']      
                                                                                                  
 block4b_project_conv (Conv2D)  (None, 32, 32, 128)  65536       ['block4b_se_excite[0][0]']      
                                                                                                  
 block4b_p

 block4d_se_excite (Multiply)   (None, 32, 32, 512)  0           ['block4d_activation[0][0]',     
                                                                  'block4d_se_expand[0][0]']      
                                                                                                  
 block4d_project_conv (Conv2D)  (None, 32, 32, 128)  65536       ['block4d_se_excite[0][0]']      
                                                                                                  
 block4d_project_bn (BatchNorma  (None, 32, 32, 128)  512        ['block4d_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block4d_drop (Dropout)         (None, 32, 32, 128)  0           ['block4d_project_bn[0][0]']     
                                                                                                  
 block4d_a

...

                                                                                                  
 block6h_add (Add)              (None, 16, 16, 256)  0           ['block6h_drop[0][0]',           
                                                                  'block6g_add[0][0]']            
                                                                                                  
 block6i_expand_conv (Conv2D)   (None, 16, 16, 1536  393216      ['block6h_add[0][0]']            
                                )                                                                 
                                                                                                  
 block6i_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6i_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6i_e

                                                                                                  
 block6k_expand_conv (Conv2D)   (None, 16, 16, 1536  393216      ['block6j_add[0][0]']            
                                )                                                                 
                                                                                                  
 block6k_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6k_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6k_expand_activation (Act  (None, 16, 16, 1536  0          ['block6k_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6k_d

                                                                                                  
 block6m_expand_bn (BatchNormal  (None, 16, 16, 1536  6144       ['block6m_expand_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 block6m_expand_activation (Act  (None, 16, 16, 1536  0          ['block6m_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6m_dwconv2 (DepthwiseConv  (None, 16, 16, 1536  13824      ['block6m_expand_activation[0][0]
 2D)                            )                                ']                               
                                                                                                  
 block6m_b

                                                                                                  
 block6o_expand_activation (Act  (None, 16, 16, 1536  0          ['block6o_expand_bn[0][0]']      
 ivation)                       )                                                                 
                                                                                                  
 block6o_dwconv2 (DepthwiseConv  (None, 16, 16, 1536  13824      ['block6o_expand_activation[0][0]
 2D)                            )                                ']                               
                                                                                                  
 block6o_bn (BatchNormalization  (None, 16, 16, 1536  6144       ['block6o_dwconv2[0][0]']        
 )                              )                                                                 
                                                                                                  
 block6o_a

ValueError: No such layer: rescaling. Existing layers are: ['input_2', 'rescaling_1', 'stem_conv', 'stem_bn', 'stem_activation', 'block1a_project_conv', 'block1a_project_bn', 'block1a_project_activation', 'block1a_add', 'block1b_project_conv', 'block1b_project_bn', 'block1b_project_activation', 'block1b_drop', 'block1b_add', 'block2a_expand_conv', 'block2a_expand_bn', 'block2a_expand_activation', 'block2a_project_conv', 'block2a_project_bn', 'block2b_expand_conv', 'block2b_expand_bn', 'block2b_expand_activation', 'block2b_project_conv', 'block2b_project_bn', 'block2b_drop', 'block2b_add', 'block2c_expand_conv', 'block2c_expand_bn', 'block2c_expand_activation', 'block2c_project_conv', 'block2c_project_bn', 'block2c_drop', 'block2c_add', 'block2d_expand_conv', 'block2d_expand_bn', 'block2d_expand_activation', 'block2d_project_conv', 'block2d_project_bn', 'block2d_drop', 'block2d_add', 'block3a_expand_conv', 'block3a_expand_bn', 'block3a_expand_activation', 'block3a_project_conv', 'block3